In [144]:
import pickle
import difflib
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [145]:
!gdown https://drive.google.com/file/d/1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac/view?usp=sharing

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac/view?usp=sharing
To: /content/view?usp=sharing
84.7kB [00:00, 56.0MB/s]


In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
df = pd.read_csv('/content/drive/MyDrive/MSIB/wisata_bandung.csv')
df

,category,tags,image,price,distance,rating,place_name,city,description
0,Cagar Alam,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",https://akcdn.detik.net.id/visual/2017/12/26/7...,30000,jauh,4.5,Gunung Tangkuban Perahu,"Cikahuripan, Kec. Lembang, Kabupaten Bandung B...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Budaya,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",https://www.indonesia.travel/content/dam/indtr...,0,dekat,4.7,Jalan Braga,"Kec. Sumur Bandung, Kota Bandung, Jawa Barat",Jalan Braga adalah nama sebuah jalan utama di ...
2,Budaya,"Budaya, Dekat dari Bandung, Murah, Gedung Sate",https://upload.wikimedia.org/wikipedia/commons...,5000,dekat,4.6,Gedung Sate,"Jl. Diponegoro No.22, Citarum, Kec. Bandung We...","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Taman Hiburan,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",https://www.hu-pakuan.com/media/original/20102...,280000,dekat,4.5,Trans Studio Bandung,"Jl. Gatot Subroto No.289A, Cibangkong, Kec. Ba...",Trans Studio Bandung adalah kawasan wisata ter...
4,Cagar Alam,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",https://assets-a1.kompasiana.com/items/album/2...,15000,dekat,4.5,Taman Hutan Raya Ir. H. Djuanda,"Kompleks Tahura, Jl. Ir. H. Juanda No.99, Cibu...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...
...,...,...,...,...,...,...,...,...,...
73,Budaya,"Budaya, Bandung, Dekat dari Bandara, Gratis",https://img.bandung.go.id/images/news/2017/01/...,0,dekat,4.6,Taman Sejarah Bandung,"Jl. Aceh No.53, Babakan Ciamis, Kec. Sumur Ban...",Taman Sejarah Bandung adalah taman yang terlet...
74,Budaya,"Budaya, Bandung, Dekat dari Bandara, Gratis",https://www.sarasvati.co.id/wp-content/uploads...,0,dekat,4.4,Roemah Seni Sarasvati,"Jl. Jend. Sudirman No.137, Karanganyar, Kec. A...",Roemah Seni Sarasvati adalah sebuah galeri yan...
75,Tempat Ibadah,"Masjid, Al Jabbar, Masjid Viral Bandung, Masji...",https://images.app.goo.gl/wU68KnyGTnEpEBP77,0,jauh,4.7,Masjid Raya Al Jabbar,"Jl. Cimencrang No.14, Cimenerang, Kec. Gedebag...","Dibangun sejak 2017, Masjid Al Jabbar Bandung ..."
76,Taman Hiburan,"Taman Hiburan, Saung Mang Udjo, Mang Udjo, Ang...",https://www.google.com/url?sa=i&url=https%3A%2...,85000,dekat,4.7,Saung Angklung Udjo,"Jl. Padasuka No.118, Pasirlayung, Kec. Cibeuny...",Saung Angklung Udjo merupakan sebuah tujuan w...


In [148]:
df_filter = df[['place_name', 'tags', 'description']]
df_filter.head()

,place_name,tags,description
0,Gunung Tangkuban Perahu,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Jalan Braga,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",Jalan Braga adalah nama sebuah jalan utama di ...
2,Gedung Sate,"Budaya, Dekat dari Bandung, Murah, Gedung Sate","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Trans Studio Bandung,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",Trans Studio Bandung adalah kawasan wisata ter...
4,Taman Hutan Raya Ir. H. Djuanda,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...


In [149]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   place_name   78 non-null     object
 1   tags         78 non-null     object
 2   description  78 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


In [150]:
df.shape

(78, 9)

In [151]:
df_filter = df_filter.dropna(axis='rows')
df_filter = df_filter.fillna('unknown')
df_filter

,place_name,tags,description
0,Gunung Tangkuban Perahu,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Jalan Braga,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",Jalan Braga adalah nama sebuah jalan utama di ...
2,Gedung Sate,"Budaya, Dekat dari Bandung, Murah, Gedung Sate","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Trans Studio Bandung,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",Trans Studio Bandung adalah kawasan wisata ter...
4,Taman Hutan Raya Ir. H. Djuanda,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...
...,...,...,...
73,Taman Sejarah Bandung,"Budaya, Bandung, Dekat dari Bandara, Gratis",Taman Sejarah Bandung adalah taman yang terlet...
74,Roemah Seni Sarasvati,"Budaya, Bandung, Dekat dari Bandara, Gratis",Roemah Seni Sarasvati adalah sebuah galeri yan...
75,Masjid Raya Al Jabbar,"Masjid, Al Jabbar, Masjid Viral Bandung, Masji...","Dibangun sejak 2017, Masjid Al Jabbar Bandung ..."
76,Saung Angklung Udjo,"Taman Hiburan, Saung Mang Udjo, Mang Udjo, Ang...",Saung Angklung Udjo merupakan sebuah tujuan w...


## Pemodelan

In [152]:
df_Tags = df.tags
df_Tags

0     Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...
1     Budaya, Dekat dari Bandung, Budaya Gratis, Bra...
2        Budaya, Dekat dari Bandung, Murah, Gedung Sate
3     Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...
4     Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...
                            ...                        
73          Budaya, Bandung, Dekat dari Bandara, Gratis
74          Budaya, Bandung, Dekat dari Bandara, Gratis
75    Masjid, Al Jabbar, Masjid Viral Bandung, Masji...
76    Taman Hiburan, Saung Mang Udjo, Mang Udjo, Ang...
77    Cagar Alam, Ranca Upas, Camping Ground, Glampi...
Name: tags, Length: 78, dtype: object

In [153]:
def remove_brackets(data):
    data = str(data).lower()
    data = data.replace(',', '')
    data = data.replace("'", '')
    return data

df_Tags = df_Tags.apply(remove_brackets)
df_Tags

0      cagar alam sedang gunung gunung tangkuban parahu
1     budaya dekat dari bandung budaya gratis braga ...
2           budaya dekat dari bandung murah gedung sate
3     taman hiburan dekat dari bandung mahal tsm tra...
4     cagar alam bandung dekat murah taman hutan ray...
                            ...                        
73             budaya bandung dekat dari bandara gratis
74             budaya bandung dekat dari bandara gratis
75    masjid al jabbar masjid viral bandung masjid d...
76    taman hiburan saung mang udjo mang udjo angklu...
77    cagar alam ranca upas camping ground glamping ...
Name: tags, Length: 78, dtype: object

In [154]:
vector_tfidf = TfidfVectorizer()
result_tfidf = vector_tfidf.fit_transform(df_Tags)

result_arr = result_tfidf.toarray()
result_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [155]:
vector_tfidf.get_feature_names_out()

array(['ade', 'africa', 'afrika', 'air', 'al', 'alam', 'amazing', 'anak',
       'and', 'angklung', 'angkringan', 'art', 'artha', 'asia', 'atas',
       'baduga', 'bakery', 'balai', 'bambu', 'bandara', 'bandung', 'batu',
       'belanda', 'bermain', 'binatang', 'bintang', 'braga', 'budaya',
       'bukit', 'bunga', 'cafe', 'cagar', 'camping', 'cepat', 'china',
       'cikole', 'citatah', 'city', 'citylight', 'ciumbuleuit', 'ciwidey',
       'creative', 'curug', 'dago', 'dari', 'dekat', 'dewi', 'dimensi',
       'djuanda', 'dreampark', 'dunia', 'dusun', 'ecopark', 'edelweis',
       'estetik', 'farm', 'feeding', 'forest', 'fossil', 'foto', 'garden',
       'gedung', 'geologi', 'glamping', 'goa', 'gondola', 'gratis',
       'great', 'ground', 'gua', 'gunung', 'haji', 'hall', 'harimau',
       'hewan', 'hiburan', 'hobbit', 'house', 'hutan', 'ibadah',
       'interaktif', 'ir', 'irma', 'istana', 'jabbar', 'jalan', 'jauh',
       'jembatan', 'jepang', 'kafe', 'kamil', 'kampung', 'kawah',
  

In [156]:
# choose one or more category
test = 'alam terdekat gratis'

In [157]:
test = 'alam sedang'
test_arr = vector_tfidf.transform([test])
test_arr = test_arr.toarray()
test_arr

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.58772028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [158]:
result = {}
for id, vector in enumerate(result_arr):
    cosine_val = cosine_similarity([result_arr[id]], test_arr)
    result[id] = cosine_val

result_desc = dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

top_n = 5

for n, place_id in enumerate(result_desc):
    if n > 4:
        break
    print(df_filter.iloc[place_id]['place_name'])

Bukit Bintang
Curug Cipanas
Jendela Alam
Selasar Sunaryo Art Space
Gunung Tangkuban Perahu


##save data / model


In [164]:
# Save dataframe
pickle.dump(result_arr, open('resulttfidf_wisata.pkl', 'wb'))

In [165]:
resulttfidf_wisata = pickle.load(open('resulttfidf_wisata.pkl','rb'))
resulttfidf_wisata

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [166]:
pickle.dump(df, open('df.pkl', 'wb'))

In [167]:
pickle.dump(vector_tfidf, open('vector_tfidf.pkl', 'wb'))

In [168]:
vectortfidf_wisata = pickle.load(open('vector_tfidf.pkl','rb'))
vectortfidf_wisata

TfidfVectorizer()